In [1]:
# エラーが出る
import MeCab

In [1]:
!curl -kL https://bootstrap.pypa.io/get-pip.py | python3

!apt-get update -y
!apt-get upgrade -y
# 👆について、十分な空き容量がないとエラーになる場合がある

!apt-get -yV install swig-doc
!apt-get -yV install swig-examples
!apt-get -yV install swig2.0-doc
!apt-get -yV install swig2.0-
!apt-get -yV install swig2.0 
!apt-get -yV install swig

!apt-get install mecab libmecab-dev mecab-ipadic-utf8 -y

!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git /tmp/work/
# 👆のclone先をホストOSとの共有ディレクトリにしているとエラーとなる場合がある。
# そのためtmp/workに保存している

!mkdir /var/lib/mecab/dic/mecab-ipadic-neologd

!apt-get install file -y

!/tmp/work/bin/install-mecab-ipadic-neologd -n -p /var/lib/mecab/dic/mecab-ipadic-neologd -y

!pip install mecab-python3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1669k  100 1669k    0     0  1085k      0  0:00:01  0:00:01 --:--:-- 1086k
     |████████████████████████████████| 1.4MB 1.2MB/s eta 0:00:01
  Found existing installation: pip 8.1.1
    Uninstalling pip-8.1.1:
      Successfully uninstalled pip-8.1.1
取得:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
取得:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB] 
取得:3 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [857 kB]
取得:4 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]        
取得:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]      
取得:6 http://archive.ubuntu.com/ubuntu xenial/main amd64 Packages [1,558 kB]
取得:7 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
取得:8 http://security.ubuntu.com/ubunt

取得:37 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libdbus-1-3 amd64 1.10.6-1ubuntu3.4 [161 kB]
取得:38 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python-apt-common all 1.1.0~beta1ubuntu0.16.04.4 [16.0 kB]
取得:39 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3-apt amd64 1.1.0~beta1ubuntu0.16.04.4 [138 kB]
取得:40 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libtiff5-dev amd64 4.0.6-1ubuntu0.6 [269 kB]
取得:41 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libtiffxx5 amd64 4.0.6-1ubuntu0.6 [9,456 B]
取得:42 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libtiff5 amd64 4.0.6-1ubuntu0.6 [149 kB]
取得:43 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgdk-pixbuf2.0-dev amd64 2.32.2-1ubuntu1.6 [44.3 kB]
取得:44 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgdk-pixbuf2.0-0 amd64 2.32.2-1ubuntu1.6 [159 kB]
取得:45 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgdk-pixbuf2.0-com

systemd-sysv (229-4ubuntu21.21) を設定しています ...
(データベースを読み込んでいます ... 現在 25300 個のファイルとディレクトリがインストールされています。)
.../libelf1_0.165-3ubuntu1.2_amd64.deb を展開する準備をしています ...
libelf1:amd64 (0.165-3ubuntu1.2) で (0.165-3ubuntu1.1 に) 上書き展開しています ...
.../libglib2.0-data_2.48.2-0ubuntu4.2_all.deb を展開する準備をしています ...
libglib2.0-data (2.48.2-0ubuntu4.2) で (2.48.2-0ubuntu4.1 に) 上書き展開しています ...
.../libglib2.0-dev_2.48.2-0ubuntu4.2_amd64.deb を展開する準備をしています ...
libglib2.0-dev (2.48.2-0ubuntu4.2) で (2.48.2-0ubuntu4.1 に) 上書き展開しています ...
.../libglib2.0-bin_2.48.2-0ubuntu4.2_amd64.deb を展開する準備をしています ...
libglib2.0-bin (2.48.2-0ubuntu4.2) で (2.48.2-0ubuntu4.1 に) 上書き展開しています ...
.../libglib2.0-0_2.48.2-0ubuntu4.2_amd64.deb を展開する準備をしています ...
libglib2.0-0:amd64 (2.48.2-0ubuntu4.2) で (2.48.2-0ubuntu4.1 に) 上書き展開しています ...
.../libdb5.3_5.3.28-11ubuntu0.2_amd64.deb を展開する準備をしています ...
libdb5.3:amd64 (5.3.28-11ubuntu0.2) で (5.3.28-11ubuntu0.1 に) 上書き展開しています ...
libc-bin (2.23-0ubuntu11) のトリガを処理しています ...
libdb5.3:amd64 (5.3.28-11ubuntu

swig3.0-doc (3.0.8-0ubuntu3) を設定しています ...
swig-doc (3.0.8-0ubuntu3) を設定しています ...
パッケージリストを読み込んでいます... 完了%
依存関係ツリーを作成しています                
状態情報を読み取っています... 完了
以下の追加パッケージがインストールされます:
   swig3.0 (3.0.8-0ubuntu3)
   swig3.0-examples (3.0.8-0ubuntu3)
以下のパッケージが新たにインストールされます:
   swig-examples (3.0.8-0ubuntu3)
   swig3.0 (3.0.8-0ubuntu3)
   swig3.0-examples (3.0.8-0ubuntu3)
アップグレード: 0 個、新規インストール: 3 個、削除: 0 個、保留: 0 個。
1,855 kB のアーカイブを取得する必要があります。
この操作後に追加で 10.9 MB のディスク容量が消費されます。
取得:2 http://archive.ubuntu.com/ubuntu xenial/universe amd64 swig3.0-examples all 3.0.8-0ubuntu3 [857 kB]
取得:1 http://61.26.74.218:80/pdata/004c761ccf6a6fb0/archive.ubuntu.com/ubuntu xenial/universe amd64 swig3.0 amd64 3.0.8-0ubuntu3 [995 kB]
取得:3 http://archive.ubuntu.com/ubuntu xenial/universe amd64 swig-examples all 3.0.8-0ubuntu3 [3,156 B]
1,855 kB を 5秒 で取得しました (371 kB/s)    
debconf: delaying package configuration, since apt-utils is not installed
以前に未選択のパッケージ swig3.0 を選択しています。
(データベースを読み込んでいます ... 現在 25382 個のファイルと

emitting double-array: 100% |###########################################| 
reading /usr/share/mecab/dic/juman/matrix.def ... 1509x1509
emitting matrix      : 100% |###########################################| 

done!
update-alternatives: /var/lib/mecab/dic/debian (mecab-dictionary) を提供するために自動モードで /var/lib/mecab/dic/juman を使います
mecab-ipadic (2.7.0-20070801+main-1) を設定しています ...
Compiling IPA dictionary for Mecab.  This takes long time...
reading /usr/share/mecab/dic/ipadic/unk.def ... 40
emitting double-array: 100% |###########################################| 
/usr/share/mecab/dic/ipadic/model.def is not found. skipped.
reading /usr/share/mecab/dic/ipadic/Auxil.csv ... 199
reading /usr/share/mecab/dic/ipadic/Symbol.csv ... 208
reading /usr/share/mecab/dic/ipadic/Noun.adverbal.csv ... 795
reading /usr/share/mecab/dic/ipadic/Filler.csv ... 19
reading /usr/share/mecab/dic/ipadic/Noun.others.csv ... 151
reading /usr/share/mecab/dic/ipadic/Others.csv ... 2
reading /usr/share/mecab/dic/ipadic

libc-bin (2.23-0ubuntu11) のトリガを処理しています ...
Cloning into '/tmp/work'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
Checking connectivity... done.
パッケージリストを読み込んでいます... 完了%
依存関係ツリーを作成しています                
状態情報を読み取っています... 完了
以下のパッケージが自動でインストールされましたが、もう必要とされていません:
  swig3.0-doc
これを削除するには 'apt autoremove' を利用してください。
以下の追加パッケージがインストールされます:
  libmagic1
以下のパッケージが新たにインストールされます:
  file libmagic1
アップグレード: 0 個、新規インストール: 2 個、削除: 0 個、保留: 0 個。
237 kB のアーカイブを取得する必要があります。
この操作後に追加で 4,156 kB のディスク容量が消費されます。
取得:1 http://61.26.74.218:80/data/024c2c209119281c/archive.ubuntu.com/ubuntu xenial-updates/main amd64 libmagic1 amd64 1:5.25-2ubuntu1.2 [216 kB]
取得:2 http://61.26.74.218:80/data/024c01205f1afa48/archive.ubuntu.com/ubuntu xenial-updates/main amd64 file amd64 1:5.25-2ubuntu1.2 [21.2 kB]
237 kB を 0秒 で取得しました (359 k

./../../libexec/iconv_euc_to_utf8.sh ./Noun.org.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.nai.csv 
rm ./Auxil.csv 
rm ./Symbol.csv 
rm ./Noun.adverbal.csv 
rm ./Filler.csv 
rm ./Noun.others.csv 
rm ./Others.csv 
rm ./Noun.name.csv 
rm ./Interjection.csv 
rm ./Noun.demonst.csv 
rm ./Noun.proper.csv 
rm ./Noun.number.csv 
rm ./Adnominal.csv 
rm ./Noun.csv 
rm ./Noun.adjv.csv 
rm ./Adj.csv 
rm ./Postp-col.csv 
rm ./Noun.place.csv 
rm ./Postp.csv 
rm ./Verb.csv 
rm ./Adverb.csv 
rm ./Conjunction.csv 
rm ./Prefix.csv 
rm ./Suffix.csv 
rm ./Noun.verbal.csv 
rm ./Noun.org.csv 
rm ./Noun.nai.csv 
./../../libexec/iconv_euc_to_utf8.sh ./pos-id.def 
./../../libexec/iconv_euc_to_utf8.sh ./unk.def 
./../../libexec/iconv_euc_to_utf8.sh ./matrix.def 
./../../libexec/iconv_euc_to_utf8.sh ./char.def 
./../../libexec/iconv_euc_to_utf8.sh ./feature.def 
./../../libexec/iconv_euc_to_utf8.sh ./rewrite.def 
./../../libexec/iconv_euc_to_utf8.sh ./left-id.def 
./../../libexec/iconv_euc_to_utf8.sh ./rig

 /usr/bin/install -c -m 644 ./unk.dic /var/lib/mecab/dic/mecab-ipadic-neologd/unk.dic
 /usr/bin/install -c -m 644 ./left-id.def /var/lib/mecab/dic/mecab-ipadic-neologd/left-id.def
 /usr/bin/install -c -m 644 ./right-id.def /var/lib/mecab/dic/mecab-ipadic-neologd/right-id.def
 /usr/bin/install -c -m 644 ./rewrite.def /var/lib/mecab/dic/mecab-ipadic-neologd/rewrite.def
 /usr/bin/install -c -m 644 ./pos-id.def /var/lib/mecab/dic/mecab-ipadic-neologd/pos-id.def
 /usr/bin/install -c -m 644 ./dicrc /var/lib/mecab/dic/mecab-ipadic-neologd/dicrc
make[1]: ディレクトリ '/tmp/work/build/mecab-ipadic-2.7.0-20070801-neologd-20190610' から出ます

[install-mecab-ipadic-NEologd] : Install completed.
[install-mecab-ipadic-NEologd] : When you use MeCab, you can set '/var/lib/mecab/dic/mecab-ipadic-neologd' as a value of '-d' option of MeCab.
[install-mecab-ipadic-NEologd] : Usage of mecab-ipadic-NEologd is here.
Usage:
    $ mecab -d /var/lib/mecab/dic/mecab-ipadic-neologd ...

[install-mecab-ipadic-NEologd] : Fin

In [2]:
import MeCab
import sys
import MeCab
m = MeCab.Tagger ("-d /var/lib/mecab/dic/mecab-ipadic-neologd")
print(m.parse ("すもももももももものうち"))

すもももももももものうち	名詞,固有名詞,一般,*,*,*,すもももももももものうち,スモモモモモモモモノウチ,スモモモモモモモモノウチ
EOS



In [3]:
# word2vecの準備
!pip install gensim

     |████████████████████████████████| 24.2MB 1.5MB/s eta 0:00:01    |██▎                             | 1.7MB 1.6MB/s eta 0:00:15     |█████▌                          | 4.1MB 1.5MB/s eta 0:00:14     |█████▋                          | 4.2MB 1.5MB/s eta 0:00:14     |█████████▊                      | 7.3MB 1.5MB/s eta 0:00:12     |██████████▏                     | 7.7MB 1.5MB/s eta 0:00:11     |███████████████▊                | 11.8MB 1.5MB/s eta 0:00:09     |████████████████▊               | 12.7MB 1.6MB/s eta 0:00:08     |█████████████████▌              | 13.2MB 1.6MB/s eta 0:00:07     |██████████████████▌             | 13.9MB 1.7MB/s eta 0:00:07     |██████████████████▉             | 14.3MB 1.7MB/s eta 0:00:06     |████████████████████▊           | 15.7MB 700kB/s eta 0:00:13     |██████████████████████▎         | 16.8MB 1.3MB/s eta 0:00:06     |█████████████████████████       | 18.8MB 1.5MB/s eta 0:00:04     |█████████████████████████▎      | 19.1MB 1.5MB/s eta 0:00:04     |██████████

In [5]:
#分かち書き作成
# 学習済みモデルを作成するため。分かち書きファイルを作成
import MeCab
import os, glob

wakati_file = './data/wakati.txt'

def wakati(text): 
        result = []
        #各行に分ける(windowsの改行コード\r\nを想定)
        lines = text.split('\r') 
        for line in lines:
            line = line.replace('\n','')

            # 形態素解析
            tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

            for chunk in tagger.parse(line).splitlines()[:-1]:
                (surface, hinshi) = chunk.split('\t')
                if hinshi.startswith('名詞') or hinshi.startswith('動詞') or hinshi.startswith('形容詞'):
    #           文章の一番最初に出てくる名詞or動詞or形容詞について処理を実行する                
                    if '*' in hinshi.split(",")[6]:
    #                     基本形が存在しない場合、表層形を返す
                        result.append(surface)
                    else :
    #                     基本形を返す
                        result.append(hinshi.split(",")[6]) 
                elif hinshi.startswith('感動詞'):                    
                    result.append(surface)         
        return result        

def main(file):
    words = []
    file_dir = os.path.abspath(file)
    try:
        bindata = open(file_dir, 'rb').read()     
        text = bindata.decode('shift_jis')        
        words = wakati(text)
    except:        
        try:
            text = bindata.decode('utf-8')
        except:
            try:
                text = bindata.decode('cp932')
            except Exception as e:
                print('error!',e)                
                exit(0)

    with open(wakati_file, 'a', encoding='utf-8') as f:
        f.write(' '.join(words))

if __name__ == '__main__':
    if os.path.exists(wakati_file):
#         すでにわかち書きが存在する場合削除
        os.remove(wakati_file)
        print('remove -> ' + wakati_file)
#   learningフォルダ直下のテキストファイルを読み込む
    files = glob.glob('./data/learning/*.txt')
    for file in files:
        if not 'wakati' in file:
            print(file)
            main(file)

remove -> ./data/wakati.txt
./data/learning/20190202.txt
./data/learning/20190203.txt
./data/learning/20190204.txt
./data/learning/20190205.txt
./data/learning/20190206.txt
./data/learning/20190207.txt
./data/learning/20190208.txt
./data/learning/20190209.txt
./data/learning/20190210.txt
./data/learning/20190211.txt
./data/learning/20190212.txt
./data/learning/20190213.txt
./data/learning/20190214.txt
./data/learning/20190215.txt
./data/learning/20190216.txt
./data/learning/20190217.txt
./data/learning/20190218.txt
./data/learning/20190219.txt
./data/learning/20190220.txt
./data/learning/20190221.txt
./data/learning/20190222.txt
./data/learning/20190223.txt
./data/learning/20190224.txt
./data/learning/20190225.txt
./data/learning/20190226.txt
./data/learning/20190227.txt
./data/learning/20190228.txt
./data/learning/20190301.txt
./data/learning/20190302.txt
./data/learning/20190303.txt
./data/learning/20190304.txt
./data/learning/20190305.txt
./data/learning/20190306.txt
./data/learning

./data/learning/20180722.txt
./data/learning/20180723.txt
./data/learning/20180724.txt
./data/learning/20180725.txt
./data/learning/20180726.txt
./data/learning/20180727.txt
./data/learning/20180728.txt
./data/learning/20180729.txt
./data/learning/20180730.txt
./data/learning/20180731.txt
./data/learning/20180801.txt
./data/learning/20180802.txt
./data/learning/20180803.txt
./data/learning/20180804.txt
./data/learning/20180805.txt
./data/learning/20180806.txt
./data/learning/20180807.txt
./data/learning/20180808.txt
./data/learning/20180809.txt
./data/learning/20180810.txt
./data/learning/20180811.txt
./data/learning/20180812.txt
./data/learning/20180813.txt
./data/learning/20180814.txt
./data/learning/20180815.txt
./data/learning/20180816.txt
./data/learning/20180817.txt
./data/learning/20180818.txt
./data/learning/20180819.txt
./data/learning/20180820.txt
./data/learning/20180821.txt
./data/learning/20180822.txt
./data/learning/20180823.txt
./data/learning/20180103.txt
./data/learnin

./data/learning/20170311.txt
./data/learning/20170312.txt
./data/learning/20170313.txt
./data/learning/20170314.txt
./data/learning/20170315.txt
./data/learning/20170316.txt
./data/learning/20170317.txt
./data/learning/20170318.txt
./data/learning/20170319.txt
./data/learning/20170320.txt
./data/learning/20170321.txt
./data/learning/20170322.txt
./data/learning/20170323.txt
./data/learning/20170324.txt
./data/learning/20170325.txt
./data/learning/20170326.txt
./data/learning/20170327.txt
./data/learning/20170328.txt
./data/learning/20170329.txt
./data/learning/20170330.txt
./data/learning/20170331.txt
./data/learning/20170401.txt
./data/learning/20170402.txt
./data/learning/20170403.txt
./data/learning/20170404.txt
./data/learning/20170405.txt
./data/learning/20170406.txt
./data/learning/20170407.txt
./data/learning/20170408.txt
./data/learning/20170409.txt
./data/learning/20170410.txt
./data/learning/20170411.txt
./data/learning/20170412.txt
./data/learning/20170413.txt
./data/learnin

./data/learning/20160909.txt
./data/learning/20160910.txt
./data/learning/20160911.txt
./data/learning/20160912.txt
./data/learning/20160913.txt
./data/learning/20160914.txt
./data/learning/20160915.txt
./data/learning/20160916.txt
./data/learning/20160917.txt
./data/learning/20160918.txt
./data/learning/20160919.txt
./data/learning/20160920.txt
./data/learning/20160921.txt
./data/learning/20160922.txt
./data/learning/20160923.txt
./data/learning/20160924.txt
./data/learning/20160925.txt
./data/learning/20160926.txt
./data/learning/20160927.txt
./data/learning/20160928.txt
./data/learning/20160929.txt
./data/learning/20160930.txt
./data/learning/20161001.txt
./data/learning/20161002.txt
./data/learning/20161003.txt
./data/learning/20161004.txt
./data/learning/20161005.txt
./data/learning/20161006.txt
./data/learning/20161007.txt
./data/learning/20161008.txt
./data/learning/20161009.txt
./data/learning/20161010.txt
./data/learning/20161011.txt
./data/learning/20161012.txt
./data/learnin

In [6]:
# word2vecでモデル作成
from gensim.models import word2vec

wakati_file = './data/wakati.txt'
model_file = './data/daily_w2v_model.model'

if __name__ == '__main__':
    file_dir = wakati_file
    w2v_data = word2vec.LineSentence(file_dir)
    model = word2vec.Word2Vec(w2v_data, size=100, window=3, hs=1, min_count=1, sg=1)
    model.save(model_file)

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [1]:
# マルコフ辞書の作成
import MeCab
import json, os, glob

markov_file = "./data/markov_dict.json"
dic = {}

def wakati(text):
    wordlist = []
    lines = text.split('\r') 
    for line in lines:
        line = line.replace('\n','')

        # 形態素解析
        tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

        for chunk in tagger.parse(line).splitlines()[:-1]:
            (surface, hinshi) = chunk.split('\t')

            if  hinshi not  in ["BOS/EOS"]:
                if hinshi.startswith('記号'):
#                   記号を辞書に登録しない
                    pass
                elif '*' in hinshi.split(",")[6]:                    
#                   基本形が存在しない場合、表層形を返す
                    wordlist.append([surface,hinshi])
                else :
#                   基本形を返す
                    wordlist.append([hinshi.split(",")[6],hinshi])

    return wordlist

def regist_dic(wordlist):
    global dic
    w1 = ""
    w2 = ""

    # 要素が3未満の場合は、何もしない
    if len(wordlist) < 3 : return

    for w in wordlist :
        word = w[0]
        if word == "" or  word == "\r\n" or word == "\n" : continue
        # 辞書に単語を設定
        if w1 and w2 :
            set_dic(dic,w1, w2, word)
        # 文末を表す語のの場合、連鎖をクリアする
        if word == "。" or word == "?" or  word == "？" :
            w1 = ""
            w2 = ""
            continue
        # 次の前後関係を登録するために、単語をスライド
        w1, w2 = w2, word

    # 辞書を保存
    json.dump(dic, open(markov_file,"w", encoding="utf-8"))

# 辞書に単語を設定 --- (*2)
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    if w1 not in dic : dic[w1] = {}
    if w2 not in dic[w1] : dic[w1][w2] = {}
    if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
    # 単語の出現数をインクリメントする
    dic[w1][w2][w3] += 1

if __name__ == '__main__':
        if os.path.exists(markov_file):
#         すでにわかち書きが存在する場合削除
            os.remove(markov_file)
            print('remove -> ' + markov_file)
        files = glob.glob('./data/learning/*.txt')
        for file in files:
            if not 'wakati' in file:
                print(file)
                try:
                    bindata = open(file, 'rb').read()
                    text = bindata.decode('shift_jis')
                except:   
                    try:
                        text = bindata.decode('utf-8')
                    except:
                        try:
                            text = bindata.decode('cp932')
                        except Exception as e:
                            print('error!',e)  
                            exit(0)

                wordlist = wakati(text)
                regist_dic(wordlist)

./data/learning/20190202.txt
./data/learning/20190203.txt
./data/learning/20190204.txt
./data/learning/20190205.txt
./data/learning/20190206.txt
./data/learning/20190207.txt
./data/learning/20190208.txt
./data/learning/20190209.txt
./data/learning/20190210.txt
./data/learning/20190211.txt
./data/learning/20190212.txt
./data/learning/20190213.txt
./data/learning/20190214.txt
./data/learning/20190215.txt
./data/learning/20190216.txt
./data/learning/20190217.txt
./data/learning/20190218.txt
./data/learning/20190219.txt
./data/learning/20190220.txt
./data/learning/20190221.txt
./data/learning/20190222.txt
./data/learning/20190223.txt
./data/learning/20190224.txt
./data/learning/20190225.txt
./data/learning/20190226.txt
./data/learning/20190227.txt
./data/learning/20190228.txt
./data/learning/20190301.txt
./data/learning/20190302.txt
./data/learning/20190303.txt
./data/learning/20190304.txt
./data/learning/20190305.txt
./data/learning/20190306.txt
./data/learning/20190307.txt
./data/learnin

./data/learning/20180708.txt
./data/learning/20180709.txt
./data/learning/20180710.txt
./data/learning/20180711.txt
./data/learning/20180712.txt
./data/learning/20180713.txt
./data/learning/20180714.txt
./data/learning/20180715.txt
./data/learning/20180716.txt
./data/learning/20180717.txt
./data/learning/20180718.txt
./data/learning/20180719.txt
./data/learning/20180720.txt
./data/learning/20180721.txt
./data/learning/20180722.txt
./data/learning/20180723.txt
./data/learning/20180724.txt
./data/learning/20180725.txt
./data/learning/20180726.txt
./data/learning/20180727.txt
./data/learning/20180728.txt
./data/learning/20180729.txt
./data/learning/20180730.txt
./data/learning/20180731.txt
./data/learning/20180801.txt
./data/learning/20180802.txt
./data/learning/20180803.txt
./data/learning/20180804.txt
./data/learning/20180805.txt
./data/learning/20180806.txt
./data/learning/20180807.txt
./data/learning/20180808.txt
./data/learning/20180809.txt
./data/learning/20180810.txt
./data/learnin

./data/learning/20170220.txt
./data/learning/20170221.txt
./data/learning/20170222.txt
./data/learning/20170223.txt
./data/learning/20170224.txt
./data/learning/20170225.txt
./data/learning/20170226.txt
./data/learning/20170227.txt
./data/learning/20170228.txt
./data/learning/20170301.txt
./data/learning/20170302.txt
./data/learning/20170303.txt
./data/learning/20170304.txt
./data/learning/20170305.txt
./data/learning/20170306.txt
./data/learning/20170307.txt
./data/learning/20170308.txt
./data/learning/20170309.txt
./data/learning/20170310.txt
./data/learning/20170311.txt
./data/learning/20170312.txt
./data/learning/20170313.txt
./data/learning/20170314.txt
./data/learning/20170315.txt
./data/learning/20170316.txt
./data/learning/20170317.txt
./data/learning/20170318.txt
./data/learning/20170319.txt
./data/learning/20170320.txt
./data/learning/20170321.txt
./data/learning/20170322.txt
./data/learning/20170323.txt
./data/learning/20170324.txt
./data/learning/20170325.txt
./data/learnin

./data/learning/20160819.txt
./data/learning/20160820.txt
./data/learning/20160821.txt
./data/learning/20160822.txt
./data/learning/20160823.txt
./data/learning/20160824.txt
./data/learning/20160825.txt
./data/learning/20160826.txt
./data/learning/20160827.txt
./data/learning/20160828.txt
./data/learning/20160829.txt
./data/learning/20160830.txt
./data/learning/20160831.txt
./data/learning/20160901.txt
./data/learning/20160902.txt
./data/learning/20160903.txt
./data/learning/20160904.txt
./data/learning/20160905.txt
./data/learning/20160906.txt
./data/learning/20160907.txt
./data/learning/20160908.txt
./data/learning/20160909.txt
./data/learning/20160910.txt
./data/learning/20160911.txt
./data/learning/20160912.txt
./data/learning/20160913.txt
./data/learning/20160914.txt
./data/learning/20160915.txt
./data/learning/20160916.txt
./data/learning/20160917.txt
./data/learning/20160918.txt
./data/learning/20160919.txt
./data/learning/20160920.txt
./data/learning/20160921.txt
./data/learnin

In [13]:
# chatを開始
import MeCab
from gensim.models import word2vec
import json, random, os

markov_dict = "./data/markov_dict.json"
model_file = './data/daily_w2v_model.model'


def preprocessing(sentence):
    return sentence.rstrip()

def talk_markov(text): 
        # 文章整形
        if text[-1] != "." or text[-1] != "?" or text[-1] != "？" or text[-1] != "!" or text[-1] != "！":                     
            text = text[:-1]
            text += '。'

        # 形態素解析
        tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

        sentence = preprocessing(text)

        for chunk in tagger.parse(sentence).splitlines()[:-1]:
            (surface, hinshi) = chunk.split('\t')
            if hinshi.startswith('名詞') or hinshi.startswith('動詞') or hinshi.startswith('形容詞'):
#           文章の一番最初に出てくる名詞or動詞or形容詞について処理を実行する                
                if '*' in hinshi.split(",")[6]:
#                     基本形が存在しない場合、表層形を返す
                    return surface
                else :
#                     基本形を返す
                    return hinshi.split(",")[6]
            elif hinshi.startswith('感動詞'):
#           　　感動詞の場合そのまま返答
                return '@' + surface

        return '@'

def load_w2v(word):
    model = word2vec.Word2Vec.load(model_file)
    try:
        similar_words = model.wv.most_similar(positive=[word])
        return random.choice([w[0] for w in similar_words])
    except:
        return word

def word_choice(sel):
    keys = sel.keys()
    ran = random.choice(list(keys))
    return ran

def make_sentence(reply):
    if '@' in reply:
#       感動詞の場合そのまま返す
        return word.replace('@','')

    markov_dic = json.load(open(markov_dict))
    if not reply == '':
        ret = []
        try:
            top = markov_dic[reply]
            word1 = word_choice(top)
            word2 = word_choice(top[word1])
            ret.append(reply)
            ret.append(word1)
            ret.append(word2)
        except Exception as e:
#             print('error:',e)
            return 'Sorry. There is no vocabulary.'

        while True:
            word3 = word_choice(markov_dic[word1][word2])
            ret.append(word3)
            if word3 == '。':
                break
            if len(ret) >= 10:
                ret.append('。')
                break
            word1, word2 = word2, word3
        return ''.join(ret)
    else:
        return ''

if __name__ == '__main__':
    while True:
        input_message = input('you -> ')
        if input_message == 'bay' or input_message == '':
            print('bot -> ' + 'bay !')
            break
            exit(0)
        word = talk_markov(input_message)
        if not '@' == word:
            reply = load_w2v(word)
        else:
#           名詞・動詞・形容詞・感動詞以外の場合
            reply = ''
        sentence = make_sentence(reply)
        print('bot -> ' + sentence)

you -> 自己紹介をお願いします。


/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


bot -> 早帰りするたい充実するせる距離を置くかえる。
you -> 長所と短所を教えてください。


/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


bot -> Sorry. There is no vocabulary.
you -> 周りの方はあなたをどのように評価していますか？


/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


bot -> ダメ送るれるのに毎日時間を潰せるた。
you -> 何か質問はありますか？


/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


bot -> これにより短絡的だ計算だたバドミントンと。
you -> bay
bot -> bay !
